1. histogram by cnn, plot by cnn
2. plot all busses and find idling areas / times
3. 

In [ ]:
import sfmta
import pandas as pd
import os

os.environ['SHUTTLE_DB_USER'] = "postgres"
os.environ['SHUTTLE_DB_PASSWORD'] =''

In [ ]:
conn = sfmta.db_connect()
#postgres is the username

In [ ]:
license_plate = 'CP96872'

query = '''SELECT local_timestamp, location[0], location[1] 
           FROM shuttle_locations l LEFT OUTER JOIN shuttles s 
           ON l.shuttle_id=s.id WHERE s.vehicle_license_plate = (%s)
          ORDER BY local_timestamp'''

cursor = conn.cursor()
cursor.execute(query, (license_plate,))

rows = cursor.fetchall()
pd.DataFrame(rows)

---

In [ ]:
import sfmta
import json
import pandas as pd
from ipywidgets import Dropdown, HBox, VBox, Button, DatePicker, ColorPicker, IntSlider
from ipyleaflet import Map, Polyline, GeoJSON, Circle

def clear_map():
    if 'm' in globals():
        m.close()
    else:
        pass
    
def clear_lat_longs():
    if 'lat_longs' in globals():
        lat_longs = None
    else:
        pass
    
def clear_time_slider():
    if 'time_slider' in globals():
        time_slider = None
    else:
        pass

def draw_update_map(b):
    clear_map()
    clear_lat_longs()
    plate = plate_dropdown.value
    global locations
    locations = sfmta.get_points_for_shuttle(conn, plate)
    zip_locations = zip(locations['lng'], locations['lat'], locations.index)
    global lat_longs
    lat_longs = [(x[0], x[1]) for x in zip_locations]
    index_value = find_index(lat_longs, time_slider.value)
    global m
    m = draw_map(lat_longs[:index_value])
    display(m)
    
button = Button(description="Draw/Update Map")
button.on_click(draw_update_map)
time_slider = IntSlider(min=0, max=100, step=1, description='% of Data', value=100)

def find_index(lat_longs, value):
    length = len(lat_longs)
    index = int(length * value/100)
    return index

def draw_map(lat_longs):
    center = [37.79481, -122.41186]
    zoom = 12
    m = Map(center=center, zoom=zoom)
    m.layout.height = '650px'
    pl = Polyline(locations=lat_longs)
    pl.color = path_color.value
    pl.fill_color = path_color.value
    m.add_layer(pl)
    return m

plates = sfmta.get_all_shuttles(conn)['LICENSE_PLATE'].unique()
plate_dropdown = Dropdown(options=plates, description='Plate')

# Add restrictions
def show_restrictions(b):
    polygons = 'vehiclerestrictions_wgs.json'
    
    with open(polygons) as f:
        polygons_json = json.load(f)

    global geojson
    geojson = GeoJSON(data=polygons_json)
    m.add_layer(geojson)
    

# Remove restrictions
def remove_restrictions(b):
    if 'geojson' in globals():
        m.remove_layer(geojson)
    else:
        pass
    
def download_data(b):
    if 'lat_longs' in globals():
        df = pd.DataFrame(data=lat_longs)
        df.to_excel('output.xlsx')
    else:
        pass
    
button_restrictions = Button(description="Show Restrictions")
button_restrictions.on_click(show_restrictions)
    
button_remove_restrictions = Button(description="Hide Restrictions")
button_remove_restrictions.on_click(remove_restrictions)

path_color = ColorPicker(description='Color')

export_data = Button(description="Download Data")
export_data.on_click(download_data)


def top_n_stops(b):
    from collections import Counter
    count_lat_longs = Counter(lat_longs).most_common()
    for lat_long in count_lat_longs[0:10]:
        location = lat_long[0]
        circle = Circle(location=location, radius=300)
        m.add_layer(circle)
    return m

button_add_top_stops = Button(description="Display Bus Idling")
button_add_top_stops.on_click(top_n_stops)

# Remove map button needs it's own function with a single argument
def remove_map(b):
    clear_map()

button_clear_map = Button(description="Remove Map")
button_clear_map.on_click(remove_map)

#Temp placeholders
import datetime
start = datetime.date(2017,11,6)
end = datetime.date(2017,11,9)
start_date = DatePicker(description='Start',value=start)
end_date = DatePicker(description='End', value=end)
companies = sfmta.get_shuttles_companies(conn)['NAME'].unique()
company = Dropdown(options=companies, value='WeDriveU', description='Company')

VBox([HBox([company, button, button_restrictions, button_remove_restrictions]),
      HBox([plate_dropdown, export_data, button_add_top_stops, button_clear_map]),
      HBox([path_color]),
      HBox([start_date]),
      HBox([end_date]),
      HBox([time_slider]),
     ])

In [ ]:
def draw_map(lat_longs):
    center = [37.79481, -122.41186]
    zoom = 12
    m = Map(center=center, zoom=zoom)
    m.layout.height = '650px'
    pl = Polyline(locations=lat_longs)
    pl.color = path_color.value
    pl.fill_color = path_color.value
    m.add_layer(pl)
    return m

In [ ]:
center = [37.79481, -122.41186]
zoom = 12
m = Map(center=center, zoom=zoom)
m.layout.height = '650px'
pl = Polyline(locations=lat_longs)
pl.color = path_color.value
pl.fill_color = path_color.value
m.add_layer(pl)
m

In [ ]:
m.get_state

In [ ]:
import os
import pandas as pd
import psycopg2

def get_all_shuttles(connection):
    cursor = connection.cursor()
    query = 'SELECT id, vehicle_license_plate, vehicle_model, vehicle_make, vehicle_year FROM shuttles'
    cursor.execute(query)
    results = cursor.fetchall()
    return pd.DataFrame.from_records(results, columns=['ID', 'LICENSE_PLATE', 'MODEL', 'MAKE', 'YEAR']).set_index('ID')


def get_points_for_shuttle(connection, license_plate=''):
    cursor = connection.cursor()
    query = 'SELECT local_timestamp, location[0], location[1] FROM shuttle_locations l LEFT OUTER JOIN shuttles s ON l.shuttle_id=s.id WHERE s.vehicle_license_plate = (%s)'

    cursor.execute(query, (license_plate,))
    results = cursor.fetchall()
    return pd.DataFrame.from_records(results, columns=['time', 'lat', 'lng']).set_index('time')

In [ ]:
def db_connect():
    try:
        username = os.environ['SHUTTLE_DB_USER']
    except KeyError:
        username = input('DB Username: ').strip()

    try:
        password = os.environ['SHUTTLE_DB_PASSWORD']
    except KeyError:
        password = input('DB Password: ').strip()

    conn = psycopg2.connect(host='localhost', user=username, password=password, database='shuttle_database')
    return conn

In [ ]:
conn = db_connect()

In [ ]:
get_points_for_shuttle(conn, 'CP96872')

In [ ]:
get_all_shuttles(conn)

In [ ]:
import pandas as pd

In [ ]:
import psycopg2
import os

In [ ]:
cursor = conn.cursor()
cursor.execute("""SELECT * from shuttle_locations s
                           join providers p
                           on p.id = s.shuttle_id
                           limit 20;""")
rows = cursor.fetchall()
pd.DataFrame(rows)

In [ ]:
column_headers = {
    'shuttle_locations': ['shuttle_id', 'tech_provider_id', 'shuttle_company_id',
                          'local_timestamp', 'location', 'cnn']
}

In [ ]:
pd.DataFrame(rows)

In [ ]:
m

In [ ]:
block_polygons = 'StreetPolygonsWGS.json'

with open(block_polygons) as f:
    block_json = json.load(f)


block_geojson = GeoJSON(data=block_json, hover_style={'fillColor': 'red'})
m.add_layer(block_geojson)
m

In [ ]:
block_json_tmp = block_json
block_json_tmp['features'] = block_json['features'][0:100] 

block_geojson_tmp = GeoJSON(data=block_json_tmp, hover_style={'fillColor': 'red'})
m.add_layer(block_geojson_tmp)

In [ ]:
block_json_tmp

In [ ]:
target_cnn = 224201

In [ ]:
for feature in block_json_tmp['features']:
    print(feature['properties']['CNN'])

In [ ]:
###########Initial loading via csv################

# import warnings
# warnings.simplefilter('ignore')

# # Identify column headers
# path = 'expanded_shuttle_three_days.csv'

# import csv
# with open(path, newline='') as f:
#     reader = csv.reader(f)
#     row1 = next(reader)

# columns = row1


# import pandas as pd

# dtypes = {'SHUTTLE_COMPANY': str, 'RESTRICTION': str, 'ARTERIAL': str,
#           'STREET': str}
# df = pd.read_csv(path, error_bad_lines=False, names=columns, header=None, dtype=dtypes)

# # Remove 1st roow
# df = df.iloc[1:]

# # Change type to datetime
# df['TIMESTAMPLOCAL'] = pd.to_datetime(df['TIMESTAMPLOCAL'], format='%d-%b-%y %I.%M.%S.%f %p')

# df

In [ ]:

    
# # Add blocks
# def show_blocks(b):
#     blocks = 'tmp2.json'
    
#     with open(blocks) as f:
#         block_json = json.load(f)

#     global block_geojson
#     block_geojson = GeoJSON(data=block_json)
#     m.add_layer(block_geojson)

# button_blocks = Button(description="Show Blocks")
# button_blocks.on_click(show_blocks)


In [ ]:
center = [37.79481, -122.41186]
zoom = 12

m2 = Map(center=center, zoom=zoom)
m2

In [ ]:
m2.add_layer(block_geojson)
m2

In [ ]:
m2.clear_layers()

In [ ]:
#print(block_geojson)

In [ ]:
block_polygons = 'block_polygons.json'
block_shape_file = '../StreetPolygons/StreetPolygons.shp'

In [ ]:
import geopandas as gpd

In [ ]:
block_polygons_df = gpd.read_file(block_shape_file)
block_polygons_df.head()

In [ ]:
columns = ['CNN', 'geometry', 'SHAPE_AREA']
columns2 = ['geometry']
block_df = block_polygons_df[columns2]
block_df.to_file(filename='tmp2.json', driver='GeoJSON')

In [ ]:
block_polygons_df.iloc[4].geometry

In [ ]:
import fiona; fiona.supported_drivers